<h2 align = 'center'> Spark Core API Assignment # 1 </h2>
<br>
<h4> Answer the following retail_db related questions using Spark RDDs. 

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession. \
builder. \
config("spark.ui.port","0"). \
config("spark.sql.warehouse.dir","/user/itv012857/warehouse"). \
enableHiveSupport(). \
master("yarn"). \
getOrCreate()

In [3]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/*")

<b> Orders: order_id,order_date,order_customer_id,order_status </b>

In [5]:
orders_rdd.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [6]:
order_items = spark.sparkContext.textFile("/public/trendytech/retail_db/order_items/*")

<b> Order Items: order_item_id,order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price

In [8]:
order_items.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [9]:
customers = spark.sparkContext.textFile("/public/trendytech/retail_db/customers/*")

<b> Customers: customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode

In [11]:
customers.take(5)

['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725',
 '4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069',
 '5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725']

<h4> 1. Find the top 10 customer who have spent the most amount

In [12]:
order_customer = orders_rdd.map(lambda x: (int(x.split(",")[0]),int(x.split(",")[2])))

In [13]:
order_amount = order_items.map(lambda x: (int(x.split(",")[1]),float(x.split(",")[4])))

In [14]:
order_amount.take(5)

[(1, 299.98), (2, 199.99), (2, 250.0), (2, 129.99), (4, 49.98)]

In [15]:
orders_joined = order_customer.join(order_amount)

In [16]:
cust_amount = orders_joined.map(lambda x: (x[1][0],x[1][1]))

In [17]:
cust_amount.take(5)

[(8827, 49.98), (8827, 299.95), (8827, 150.0), (8827, 199.92), (2911, 179.97)]

In [18]:
agg_cust_amount = cust_amount.reduceByKey(lambda x,y: x+y)

In [19]:
agg_cust_amount.sortBy(lambda x: x[1],False).take(10)

[(791, 10524.169999999996),
 (9371, 9299.029999999999),
 (8766, 9296.14),
 (1657, 9223.71),
 (2641, 9130.92),
 (1288, 9019.11),
 (3710, 9019.099999999999),
 (4249, 8918.85),
 (5654, 8904.95),
 (5624, 8761.98)]

<h4> 2. Top 10 Product IDs with most quantities sold

<b> Order Item : order_item_id,order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price </b>

In [20]:
product_qty = order_items.map(lambda x:(int(x.split(",")[2]),int(x.split(",")[3])))

In [21]:
product_qty.take(5)

[(957, 1), (1073, 1), (502, 5), (403, 1), (897, 2)]

In [22]:
product_qty.reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1],False).take(10)

[(365, 73698),
 (502, 62956),
 (1014, 57803),
 (191, 36680),
 (627, 31735),
 (403, 22246),
 (1004, 17325),
 (1073, 15500),
 (957, 13729),
 (977, 998)]

<h4> 3. How many customers are from Caguas city?

<b> Customers : customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode

In [23]:
customers.filter(lambda x: x.split(",")[6]=='Caguas').count()

4584

<h4> 4. Top 3 States with Maximum Customers

In [24]:
customers.map(lambda x: (x.split(",")[7],1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],False).take(3)

[('PR', 4771), ('CA', 2012), ('NY', 775)]

<h4> 5. How many customer have spent more than $1000 in total?

In [25]:
# Using agg_cust_amount RDD created above as it has both customer and total amount spent

agg_cust_amount.take(5)

# Following Data is - Customer ID, Total Amount Spent

[(7276, 2489.63),
 (656, 4455.39),
 (10500, 5567.5),
 (4320, 5016.34),
 (6932, 4405.5)]

In [26]:
agg_cust_amount.filter(lambda x:x[1] > 1000).count()

11148

<h3> 6. Which State has Most Number of Orders in Closed Status?

<b> Orders: order_id,order_date,order_customer_id,order_status </b>

In [30]:
closed_orders = orders_rdd.filter(lambda x: x.split(",")[3] == 'CLOSED')

In [80]:
closed_orders_customers = closed_orders.map(lambda x:(int(x.split(",")[2]),1))

In [81]:
closed_orders_customers.take(10)

[(11599, 1),
 (8827, 1),
 (1837, 1),
 (1205, 1),
 (11441, 1),
 (9503, 1),
 (5863, 1),
 (12271, 1),
 (7073, 1),
 (4791, 1)]

<b> Customers : customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode

In [82]:
customer_state = customers.map(lambda x: (int(x.split(",")[0]),x.split(",")[7]))

In [83]:
customer_state.take(5)

[(1, 'TX'), (2, 'CO'), (3, 'PR'), (4, 'CA'), (5, 'PR')]

In [84]:
closed_orders_state = customer_state.join(closed_orders_customers)

In [85]:
closed_orders_state.take(10)

[(4, ('CA', 1)),
 (8, ('MA', 1)),
 (12, ('TX', 1)),
 (16, ('PR', 1)),
 (20, ('NJ', 1)),
 (28, ('WA', 1)),
 (32, ('PR', 1)),
 (36, ('PR', 1)),
 (40, ('CA', 1)),
 (60, ('NY', 1))]

In [86]:
closed_orders_state.map(lambda x:(x[1][0],x[1][1])).reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1],False).take(1)

[('PR', 2891)]

<h3> 7. How many customers are active? </h3>
Active customer are customers who have placed atleast one order.

In [92]:
orders_rdd.map(lambda x:x.split(",")[2]).distinct().count()

12405

<h3> 8. What is the revenue generated by each state in descending order?    

In [56]:
# Using agg_cust_amount RDD created above as it has both customer and total amount spent
# Joining it with Customer RDD to get the state

agg_cust_amount.take(10)

[(7276, 2489.63),
 (656, 4455.39),
 (10500, 5567.5),
 (4320, 5016.34),
 (6932, 4405.5),
 (356, 3159.44),
 (5200, 5679.24),
 (11116, 4799.4400000000005),
 (7604, 3752.55),
 (1780, 7071.280000000001)]

In [72]:
# Also using customer_state RDD created above as it already has Customer & State relationship
customer_state.take(5)

[(1, 'TX'), (2, 'CO'), (3, 'PR'), (4, 'CA'), (5, 'PR')]

In [73]:
customer_state_amount = customer_state.join(agg_cust_amount)

In [74]:
customer_state_amount.take(5)

[(6, ('NJ', 3259.51)),
 (12, ('TX', 6009.35)),
 (18, ('CA', 3519.6)),
 (24, ('PR', 2769.47)),
 (30, ('PR', 1105.89))]

In [77]:
customer_state_amount.map(lambda x: (x[1][0],x[1][1])).reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[1],False).collect()

[('PR', 13208867.690000003),
 ('CA', 5542722.999999998),
 ('NY', 2152706.74),
 ('TX', 1731407.4900000002),
 ('IL', 1457225.83),
 ('FL', 1048609.77),
 ('OH', 773804.1100000001),
 ('MI', 730078.9700000001),
 ('PA', 724375.9299999998),
 ('NJ', 606550.9900000001),
 ('AZ', 566459.29),
 ('GA', 467765.1800000001),
 ('MD', 456100.42000000004),
 ('NC', 378877.64),
 ('CO', 358310.60000000003),
 ('VA', 344824.35000000003),
 ('OR', 315239.51),
 ('MA', 306025.73000000004),
 ('TN', 297614.41000000003),
 ('NV', 276364.97),
 ('MO', 260417.28000000003),
 ('HI', 238208.11000000002),
 ('CT', 211264.24000000002),
 ('UT', 184356.83),
 ('LA', 182954.74000000002),
 ('NM', 181366.65),
 ('WA', 172982.45),
 ('WI', 164058.32),
 ('MN', 133183.97),
 ('SC', 129411.79000000001),
 ('IN', 110015.14000000001),
 ('DC', 108841.48000000001),
 ('KY', 94894.75000000001),
 ('KS', 85865.07),
 ('DE', 48873.25),
 ('RI', 47439.35),
 ('OK', 43493.32),
 ('WV', 43452.670000000006),
 ('ND', 41221.97),
 ('ID', 31791.199999999997),
 (